In [33]:
import numpy as np
import re
import json
#!conda install graphviz
from graphviz import Digraph
import json

In [34]:
# if graphiz does not work with you in window10
# please do the following
# environment variables from start
# edit Path 
# add the following path:C:\Users\UserName\Anaconda3\Library\bin\graphviz

In [35]:
def getOpenParent(Locations, value):
    loc = Locations[0]
    for i in range(len(Locations)):
        if Locations[i] > value:
            break
        else:
            loc = Locations[i]
    return loc


# getOpenParent([0,10,11], 6)

In [36]:
# inp = "((a|(bc*|s)|d)*e)|(i)"
def solve(ChList):
    NodeCount = 0
    NFA = []
    while len(ChList) != 1:
        index_open = np.where(ChList == '(')[0]
        index_closed = np.where(ChList == ')')[0]

        loc_C = index_closed[0]
        loc_O = getOpenParent(index_open, loc_C)

        reg_list = np.concatenate((ChList[loc_O+1 :loc_C], np.array(['|'])))    
#         print(reg_list)

        idx_list = [idx + 1 for idx, val in enumerate(reg_list) if val == '|']
        lists_split_Or = [reg_list[i: j-1] for i, j in
                zip([0] + idx_list, idx_list + ([len(reg_list)] if idx_list[-1] != len(reg_list) else []))]

#         print(lists_split_Or)

        subNFA_to_or = []
        for subList in lists_split_Or:
            subNFA = []
            for j in range(len(subList)):
                print(subList[j])
                if subList[j] == '*':
                    NFA.append([])
                    NFA[subNFA[len(subNFA)-1][0]].append((NodeCount, 'epsilon'))
                    NFA[subNFA[len(subNFA)-1][1]].append((subNFA[len(subNFA)-1][0], 'epsilon'))
                    NFA[subNFA[len(subNFA)-1][1]].append((NodeCount, 'epsilon'))
                    subNFA[len(subNFA)-1] = (subNFA[len(subNFA)-1][0], NodeCount)
                    NodeCount += 1

                elif type(subList[j]) == tuple:
                    subNFA.append(subList[j])

                else:
                    NFA.append([])
                    NFA.append([])
                    NFA[NodeCount].append((NodeCount+1, subList[j]))
                    subNFA.append((NodeCount, NodeCount+1))
                    NodeCount += 2

#             print(subNFA)

            #to handle anding each sub graph
            for i in range(1, len(subNFA)):
                NFA[subNFA[i-1][1]].append((subNFA[i][0], "epsilon"))

            subNFA_to_or.append((subNFA[0][0], subNFA[len(subNFA)-1][1]))

        #to handle oring each sub graph
        NFA.append([])
        NFA.append([])
        for tup in subNFA_to_or:
            NFA[NodeCount].append((tup[0], "epsilon"))
            NFA[tup[1]].append((NodeCount+1, "epsilon"))   

        ChList[loc_O] = (NodeCount, NodeCount+1)
        NodeCount += 2 

        ChList = np.delete(ChList, [i for i in range(loc_O+1,loc_C+1)])
#         print(ChList)
    print(NFA)
    return NFA, ChList[0][0], ChList[0][1]

In [80]:
def draw(res,start,end):
        json_file={}
       
        #draw the graph 
        f = Digraph('NFA Graph',format="png")
        f.attr(rankdir='LR', size='8,5')
        
        f.attr('node', shape='doublecircle')
        f.node('S'+str(end)) 
        f.attr('node', shape='circle')
        json_file["startingState"]="S"+str(start)
        f.node('S'+str(start),color="blue")
        #f.node('S'+str(start),color="red")
        for node_inx in range(len(res)):
            terminate=False
            if node_inx==end:
                terminate=True
            curr_state="S"+str(node_inx)
            json_file[curr_state]={
                "isTerminatingState":terminate}
            for edge in res[node_inx]:      
                out_node=str(edge[0])
                inp_char=edge[1]
                if inp_char in json_file[curr_state]:
                     json_file[curr_state][inp_char].append("S"+out_node)
                else:
                    json_file[curr_state][inp_char] = ["S"+out_node]
                
                #draw
                f.edge('S'+str(node_inx), 'S'+str(out_node), label=inp_char)
            
            json_object=json.dumps(json_file, indent=4)
            # Writing to sample.json
            with open("nfa.json", "w") as outfile:
                outfile.write(json_object)
        
        f.view()
        return json_object

In [85]:
def main():
    reg_exp = input("enter the regular expression:   ")
    try:
        re.compile(reg_exp)
    except re.error:
        print("Non valid regex pattern")
        exit()
        
    inp = '('+reg_exp+')'
    ChList = np.array([char for char in inp],dtype=object)  
    
    NFA, start, end = solve(ChList)
    #output json file and draw the graph
    lis=list(range(len(NFA)))
    print(lis)
    for node_inx in range(len(NFA)):
        for edge in NFA[node_inx]:
            if edge[0] in lis:
                lis.remove(edge[0])
    print(lis)
    end=len(NFA)-1
    start=lis[0]
    
    json_object= draw(NFA,start,end)
    print("\n\n",json_object)

In [86]:
main()

enter the regular expression:   a|b
a
b
[[(1, 'a')], [(5, 'epsilon')], [(3, 'b')], [(5, 'epsilon')], [(0, 'epsilon'), (2, 'epsilon')], []]
[0, 1, 2, 3, 4, 5]
[4]


 {
    "startingState": "S4",
    "S0": {
        "isTerminatingState": false,
        "a": [
            "S1"
        ]
    },
    "S1": {
        "isTerminatingState": false,
        "epsilon": [
            "S5"
        ]
    },
    "S2": {
        "isTerminatingState": false,
        "b": [
            "S3"
        ]
    },
    "S3": {
        "isTerminatingState": false,
        "epsilon": [
            "S5"
        ]
    },
    "S4": {
        "isTerminatingState": false,
        "epsilon": [
            "S0",
            "S2"
        ]
    },
    "S5": {
        "isTerminatingState": true
    }
}


In [8]:
########### another attempts ##############3

def solve(inp,global_start):
    #inp = "(a|(a(b|(Mn|Yk|C*|vxz)n))|c)*(ac)"
    ChList = np.array([char for char in inp])
    
    # while len(ChList) != 1:
    index_open = np.where(ChList == '(')[0]
    index_closed = np.where(ChList == ')')[0]
    subNFA = []
    if len(index_closed) != 0:
        loc_C = index_closed[0]
        loc_O = getOpenParent(index_open, loc_C)
        #print(loc_C)
        
        list_split_Or = inp[loc_O+1: loc_C].split('|') #split the substr by or
        list_split_Or.sort(key = len, reverse = True)
        #print(list_split_Or)
        
        k = 1  #start node
        end_node = max([len(i.replace('*', '')) for i in list_split_Or]) + 1
        #print(end_node)
        global_start +=1
        #subNFA = []
        for subList in list_split_Or:
            for j in range(len(subList)): 
                
                if (j == len(subList)-1 and (subList[j].isalpha() or subList[j].isnumeric())) or (j == len(subList)-2 and (subList[j].isalpha() or subList[j].isnumeric()) and subList[j+1] == '*'):
                    if j == 0: subNFA.append((1, end_node, subList[j]))
                    else: subNFA.append((k, end_node, subList[j]))
                    k += 1
                    continue
                if j == len(subList)-1 and subList[j] == '*':
                    if j == 1:
                        subNFA.append((1, end_node, 'epsilon'))
                        subNFA.append((end_node, 1, 'epsilon'))
                    else:    
                        subNFA.append((k, end_node, 'epsilon'))
                        subNFA.append((end_node, k, 'epsilon'))
                    continue                
                    
                if subList[j].isalpha() or subList[j].isnumeric():
                    if j == 0:
                        subNFA.append((1, k+1, subList[j]))
                    else:
                        subNFA.append((k, k+1, subList[j]))
                    k += 1
                    
                elif subList[j] == '*':
                    if j == 1:
                        subNFA.append((1, k, 'epsilon'))
                        subNFA.append((k, 1, 'epsilon'))
                    else:    
                        subNFA.append((k-1, k, 'epsilon'))
                        subNFA.append((k, k-1, 'epsilon'))
               
                        
    return (global_start,global_start+1,subNFA),loc_O, loc_C
    #return subNFA,loc_O, loc_C


def Oring(states):
    NFA=[]
    for state in states:
        NFA.append(state)

def main():
    
    reg_exp = input("enter the regular expression:   ")
    try:
        re.compile(reg_exp)
    except re.error:
        print("Non valid regex pattern")
    #     exit()

    dic={}
    #inp = '(a|(a(b|(Mn|Yk|C*|vxz)n))|c)*(ac)'
    global_char=2
    substring=reg_exp
    ChList = np.array([char for char in reg_exp])
    global_start=0
    index_open = np.where(ChList == '(')[0]
    for i in range(len(index_open)):
        res,start,end=solve(substring, global_start)
        print(res)
        dic[global_char]=res
        string_list = list(substring)   
        string_list[start:end+1] = str(global_char)
        global_char +=1
        global_start +=2
        new_string = "".join(string_list)
        substring = new_string
        print(new_string)
        
    print("\n")
    for key in dic:
        print(key,":",dic[key])
        print("\n")
    NFA=[]
   # for key in dic:
    #   NFA.append((1, key, 'epsilon'))

    #print("here NFA came")   
    
    #output to json file 
   # json_file={}
    #for key in dic:
    #   NFA.append((1, key, 'epsilon'))

    #for key in dic:
    #    for i in range(len(dic[key][2])):
    #        if dic[key][2][i][2].isnumeric():
    #            print(dic[key][2][i])
    #            num = int(dic[key][2][i][2])
    #            dic[key][2][i][2] = dic[key][2][i][2][:2] + (dic[num],) + dic[key][2][i][2][2 + 1:]
               # dic[key][2][i][2] = dic[num]
                #dic[key][2][i]=dic[dic[key][2][i][2]]
            #print(state)

    
if __name__ == '__main__':
    main()

KeyboardInterrupt: Interrupted by user